In [2]:
import numpy as np
import os
import cv2
import multiprocessing as mp
from copy import deepcopy

import pyspark

import os

os.environ["PYSPARK_PYTHON"]="/usr/bin/python3.6"
os.environ["PYSPARK_DRIVER_PYTHON"]="/usr/bin/ipython3"
os.environ["SPARK_YARN_USER_ENV"]="PYSPARK_PYTHON=/usr/bin/python3.6"
"""
export PYSPARK_PYTHON=/usr/bin/python3.6
export PYSPARK_DRIVER_PYTHON=/usr/bin/ipython3
export SPARK_YARN_USER_ENV="PYSPARK_PYTHON=/usr/bin/python3.6"
"""

'\nexport PYSPARK_PYTHON=/usr/bin/python3.6\nexport PYSPARK_DRIVER_PYTHON=/usr/bin/ipython3\nexport SPARK_YARN_USER_ENV="PYSPARK_PYTHON=/usr/bin/python3.6"\n'

In [3]:
from pyspark.accumulators import AccumulatorParam
class Matrix3DAccumulatorParam(AccumulatorParam):
	def zero(self, mInitial):
		aaZeros = np.zeros(mInitial.shape)
		return aaZeros

	def addInPlace(self, mAdd, lIndex):
		if type(lIndex) == list:
			mAdd[lIndex[0], lIndex[1], lIndex[2]] += 1
		else:
			mAdd += lIndex
		return mAdd

In [4]:
## Functions to Aid on the Parallelization Task
def gaussian_smoothing(input_img):
								
	gaussian_filter=np.array([[0.109,0.111,0.109],
							  [0.111,0.135,0.111],
							  [0.109,0.111,0.109]])
								
	return cv2.filter2D(input_img,-1,gaussian_filter) 

def canny_edge_detection(input):
	
	input = input.astype('uint8')

	# Using OTSU thresholding - bimodal image
	otsu_threshold_val, ret_matrix = cv2.threshold(input,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
	
	#lower_threshold = otsu_threshold_val * 0.8
	#upper_threshold = otsu_threshold_val * 1.7
	
	lower_threshold = otsu_threshold_val * 0.4
	upper_threshold = otsu_threshold_val * 1.3
	
	print(lower_threshold,upper_threshold)
	
	#print(lower_threshold,upper_threshold)
	edges = cv2.Canny(input, lower_threshold, upper_threshold)
	return edges

def HoughCircles(input,circles): 
	rows = input.shape[0] 
	cols = input.shape[1] 
	
	# initializing the angles to be computed 
	sinang = dict() 
	cosang = dict() 
	
	# initializing the angles  
	for angle in range(0,360): 
		sinang[angle] = np.sin(angle * np.pi/180) 
		cosang[angle] = np.cos(angle * np.pi/180) 
			
	# initializing the different radius
	# For Test Image <----------------------------PLEASE SEE BEFORE RUNNING------------------------------->
	#radius = [i for i in range(10,70)]
	#For Generic Images
	length=int(rows/2)
	radius = [i for i in range(5,length)]

	
	# Initial threshold value 
	threshold = 190 
	
	for r in radius:
		#Initializing an empty 2D array with zeroes 
		acc_cells = np.full((rows,cols),fill_value=0,dtype=np.uint64)
		 
		# Iterating through the original image 
		for x in range(rows): 
			for y in range(cols): 
				#print (x)
				#print (y)
				if input[x][y] == 255:# edge 
					# increment in the accumulator cells 
					for angle in range(0,360): 
						b = int(y - int(round(r * sinang[angle])))
						a = int(x - int(round(r * cosang[angle])))
						if a >= 0 and a < rows and b >= 0 and b < cols: 
							acc_cells[a][b] += 1
							 
		print('For radius: ',r)
		acc_cell_max = np.amax(acc_cells)
		print('max acc value: ',acc_cell_max)
		
		if(acc_cell_max > 150):  

			print("Detecting the circles for radius: ",r)	   
			
			# Initial threshold
			acc_cells[acc_cells < 150] = 0  
			   
			# find the circles for this radius 
			for i in range(rows): 
				for j in range(cols): 
					if(i > 0 and j > 0 and i < rows-1 and j < cols-1 and acc_cells[i][j] >= 150):
						avg_sum = np.float32((acc_cells[i][j]+acc_cells[i-1][j]+acc_cells[i+1][j]+acc_cells[i][j-1]+acc_cells[i][j+1]+acc_cells[i-1][j-1]+acc_cells[i-1][j+1]+acc_cells[i+1][j-1]+acc_cells[i+1][j+1])/9) 
						#print("Intermediate avg_sum: ",avg_sum)
						if(avg_sum >= 33):
							#print("For radius: ",r,"average: ",avg_sum,"\n")
							circles.append((i,j,r))
							acc_cells[i:i+5,j:j+7] = 0

In [5]:
def mapping (window):
	global input_broad, acc_cells, edged_broad
	input = input_broad.value
	edged_image = edged_broad.value

	print('Mapping for Window:')
	print(window[0], window[2])
	rows = window[1] - window[0] 
	cols = window[3] - window[2]
	
	# initializing the angles to be computed 
	sinang = dict() 
	cosang = dict() 
	
	# initializing the angles  
	for angle in range(0,360): 
		sinang[angle] = np.sin(angle * np.pi/180) 
		cosang[angle] = np.cos(angle * np.pi/180) 
			
	# initializing the different radius
	# For Test Image <----------------------------PLEASE SEE BEFORE RUNNING------------------------------->
	#radius = [i for i in range(10,70)]
	#For Generic Images
	length=min(rows, cols)
	radius = [i for i in range(5,length)]

	
	# Initial threshold value 
	threshold = 190 
	
	for r in radius:
		#Initializing an empty 2D array with zeroes 
		 
		# Iterating through the original image 
		for x in range(window[0], window[1]): 
			for y in range(window[2], window[3]): 
				#print (x)
				#print (y)
				if x >= 0 and x < input.shape[0] and y >=0 and y < input.shape[1] and edged_image[x][y] == 255:# edge 
					# increment in the accumulator cells 
					for angle in range(0,360): 
						b = int(y - int(round(r * sinang[angle])))
						a = int(x - int(round(r * cosang[angle])))
						if a >= 0 and a < input.shape[0] and b >= 0 and b < input.shape[1]: 
							acc_cells.add([r, a, b])

In [6]:
def reduce (r):
	global acc_cells, circles
	acc_cell_max = np.amax(acc_cells.value[r])
	cells_val = acc_cells.value
	
	if(acc_cell_max > 150):  
		print("Detecting the circles for radius: ",r)	   
		
		# Initial threshold
		cells_val[r][cells_val[r] < 150] = 0  
		
		# find the circles for this radius 
		for i in range(rows): 
			for j in range(cols): 
				if(i > 0 and j > 0 and i < rows-1 and j < cols-1 and cells_val[r][i][j] >= 150):
					avg_sum = np.float32((cells_val[r][i][j]+cells_val[r][i-1][j]+cells_val[r][i+1][j]+cells_val[r][i][j-1]+cells_val[r][i][j+1]+cells_val[r][i-1][j-1]+cells_val[r][i-1][j+1]+cells_val[r][i+1][j-1]+cells_val[r][i+1][j+1])/9) 
					#print("Intermediate avg_sum: ",avg_sum)
					if(avg_sum >= 33):
						#print("For radius: ",r,"average: ",avg_sum,"\n")
						circles.append((i,j,r))
						#cells_val[r, i:i+5,j:j+7] = 0

In [7]:
def applyHoughCircleParallel(imgFile):
	#3. Detect Circle radius
	#4. Perform Circle Hough Transform
	global sc, input, circles
	windows = []
	rows = input.shape[0] 
	cols = input.shape[1]
	
	print('Apply Hough Transform')
	# cv2.imshow('Circle Detected Image',edged_image)
    # Define the window size
	windowsize_r = 50
	windowsize_c = 50
	length=min(min(windowsize_r, windowsize_c), min(rows, cols))
	radius = [i for i in range(5,length)]

	print('Chopping Windows')
    # Crop out the window and calculate the histogram
	for r in range(0,input.shape[0], windowsize_r):
		for c in range(0,input.shape[1], windowsize_c):
			window = (r, r+windowsize_r, c, c+windowsize_c)
			windows.append(window)
    
	print('Parallelizing')
	#for window in windows:
		#mapping(window)
	sc.parallelize(windows).foreach(mapping)

	print('Detecting Circles')
	# Detect Circle 
	#HoughCircles(edged_image,circles)
	for r in radius:
		reduce(r)
	
	# Print the output
	for vertex in circles:
		cv2.circle(orig_img,(vertex[1],vertex[0]),vertex[2],(0,255,0),1)
		cv2.rectangle(orig_img,(vertex[1]-2,vertex[0]-2),(vertex[1]-2,vertex[0]-2),(0,0,255),3)
        
	print('Saving Detected Circles in image')
	#cv2.imshow('Circle Detected Image',orig_img) 
	cv2.imwrite(imgFile, orig_img)

In [ ]:
# Get First Image
imagePath = os.path.join('images', 'mini.jpg')
orig_img = cv2.imread(imagePath)

input = cv2.imread(imagePath,cv2.IMREAD_GRAYSCALE)
    
# Create copy of the orignial image
input_img = deepcopy(input)

#Steps
#1. Denoise using Gaussian filter and detect edges using canny edge detector
smoothed_img = gaussian_smoothing(input_img)

#2. Detect Edges using Canny Edge Detector
edged_image = canny_edge_detection(smoothed_img)

circles = []
rows = input.shape[0] 
cols = input.shape[1]
length=min(rows, cols)

sc = pyspark.SparkContext.getOrCreate()
acc_cells = sc.accumulator(np.zeros((length + 1, rows, cols)), Matrix3DAccumulatorParam())
input_broad = sc.broadcast(input)
edged_broad = sc.broadcast(edged_image)

applyHoughCircleParallel ('mini.jpg')

sc.stop()

36.4 118.3
Apply Hough Transform
Chopping Windows
Parallelizing
